In [1]:
# path_to_prototype = '../'
import pandas as pd
import sys
# sys.path.append(path_to_prototype)
import pickle as pkl
from __future__ import print_function
from IKOC import IKOC
from IKOC import KOC
from sklearn.model_selection import KFold
import math
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from ipyparallel import Client
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import rbf_kernel
from functools import partial
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### For AUC Calculation
from sklearn import metrics

C:\Users\Workstation\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Loading Extracted Features

In [2]:

tot_dat = pd.read_csv('heart_data.csv', index_col=0)
labels = pd.read_csv('heart_labels.csv', index_col=0)
priv_dat = pd.read_csv('priv_heart_data.csv', index_col=0)


# print tot_dat.shape, labels.shape, priv_dat.shape
# target_files = labels.query('Class == {}'.format(index)).index
# target_labels = labels.ix[target_files].Class == index

# test_indices = set(feature_space.index) - set(target_files) ### in Set for
# test_indices = pd.Index(test_indices)
# Outlier_labels = labels.ix[test_indices].Class == index
# print Outlier_labels

In [3]:
### Normalized the features
normalized = StandardScaler().fit_transform(tot_dat)
feature_space = pd.DataFrame(normalized, 
                             index=tot_dat.index, 
                             columns=tot_dat.columns)

normalized = StandardScaler().fit_transform(priv_dat)
privileged_space_tot = pd.DataFrame(normalized,
                               index=priv_dat.index,
                               columns=priv_dat.columns)

#### Select which feature would be selected as Privileged Feature and remove this feature from the feature space
#### (Privileged feature removal part is in the next cell) 

### Select sex as privileged attribute i.e p1
# privileged_space = privileged_space_tot.ix[:]['p1']

### Select age as privileged attribute i.e p2
privileged_space = privileged_space_tot.ix[:]['p2']

### Select elect as privileged attribute i.e p3
# privileged_space = privileged_space_tot.ix[:]['p3']

#### Ordinary KOC(KRR-based One-class Classification) or OneClassELM with Thr1 i.e. some % of rejection

In [4]:
#### Just remove the group attribute because we are treating it as a privileged information

### Drop sex because it is privileged attribute as p1
# feature_space = feature_space.drop('a2', axis=1)
# privfeat = 'Sex'

### Drop age because it is privileged attribute as p2
feature_space = feature_space.drop('a1', axis=1)
privfeat = 'Age'

### Drop elect because it is privileged attribute as p3
# feature_space = feature_space.drop('a7', axis=1)
# privfeat = 'Elect'

# feature_space

In [5]:
all_res_val = {}
all_res_test = {}
X_train = {}
X_testN = {}
y_train = {}
y_testN = {}
X_val = {}  
X_testO = {}
y_val = {}
y_testO = {} 

##### Either run the commented lines after first for loop to generate 5-fold indices or load the saving indices as below
with open('heart_5fold_index.pkl', 'rb') as f:
    X_train, X_testN, y_train, y_testN, X_val, X_testO, y_val, y_testO = pkl.load(f)
        
for index in range(1, 3):
    
        ##### Either run this to generate 5-fold indices or load the saving indices as before above for loop
        
#     target_files = labels.query('Class == {}'.format(index)).index   ### All Target Class Files wiith indices
#     test_indices = set(feature_space.index) - set(target_files) ### in Set for
#     test_indices = pd.Index(test_indices) ### Convert set to index
#     target_labels = labels.ix[target_files].Class == index ### Target data labels
#     Outlier_labels = labels.ix[test_indices].Class == index ### Target data labels

    
#     kf = KFold(n_splits=5) ### k-fold Cross-Validation
#     # For Target Files
#     fold = 0
#     X_train[index] = {}
#     X_testN[index] = {}
#     y_train[index] = {}
#     y_testN[index] = {}
#     for train_index, test_index in kf.split(target_files):
#         #print("TRAIN:", train_index, "TEST:", test_index)
#         X_train[index][str(fold)]=target_files[train_index]
#         X_testN[index][str(fold)]=target_files[test_index]
#         y_train[index][str(fold)]=target_labels[train_index]
#         y_testN[index][str(fold)]=target_labels[test_index]
#         fold = fold+1

#     # For Outlier Files
#     fold = 0
#     X_val[index] = {}  
#     X_testO[index] = {}
#     y_val[index] = {}
#     y_testO[index] = {}    
#     for train_index, test_index in kf.split(test_indices):
# #         print("TRAIN:", train_index, "TEST:", test_index)
#         X_val[index][str(fold)]=test_indices[train_index]
#         X_testO[index][str(fold)]=test_indices[test_index]
#         y_val[index][str(fold)]=Outlier_labels[train_index]
#         y_testO[index][str(fold)]=Outlier_labels[test_index] 
#         fold=fold+1

        
### Cocatenate and Prepare final data
    all_res_val[index] = {}
    all_res_test[index] = {}
    for f in range(0, 5):
#        print index,f
        X_vali = pd.concat([feature_space.ix[X_train[index][str(f)]][int(math.floor(len(X_train[index][str(f)])*0.8)):], feature_space.ix[X_val[index][str(f)]]], axis=0,sort=False)
        Y_vali = pd.concat([y_train[index][str(f)][int(math.floor(len(X_train[index][str(f)])*0.8)):], y_val[index][str(f)]], axis=0,sort=False)
        X_testi = pd.concat([feature_space.ix[X_testN[index][str(f)]], feature_space.ix[X_testO[index][str(f)]]], axis=0,sort=False)
        Y_testi = pd.concat([y_testN[index][str(f)], y_testO[index][str(f)]], axis=0,sort=False)
#### Take 80% of samples for training and keep 20% samples for validation
        train_id = X_train[index][str(f)][:int(math.floor(len(X_train[index][str(f)])*0.8))]
        train_data = feature_space.ix[train_id]
        train_label = y_train[index][str(f)][:int(math.floor(len(X_train[index][str(f)])*0.8))]
        all_nu = np.linspace(0.05, 0.7, 10)
        all_gamma = np.logspace(-10, 10, 20) # Power of 10 is called logspace. -5 is start point, 5 is last point ans 20 points will be generated
        all_res_val[index][f] = {}
        all_res_test[index][f] = {}
        for nu in all_nu:
            all_res_val[index][f][nu] = {}
            all_res_test[index][f][nu] = {}
            for gamma in all_gamma:
                features_kernel = partial(rbf_kernel, gamma=gamma)
                model = KOC(nu, features_kernel=features_kernel)
                model.fit(train_data.as_matrix())
                pred_val = model.decision_function(X_vali)
                all_res_val[index][f][nu][gamma] = pred_val              
                pred_test = model.decision_function(X_testi)
                all_res_test[index][f][nu][gamma] = pred_test           
#                 print '\r {0} {1} {2:.2f} {3:.2f}'.format(index, f, nu, gamma),
                ### Python 3
                print ('\r {0} {1} {2:.2f} {3:.2f}'.format(index, f, nu, gamma), end='')

C:\Users\Workstation\Anaconda2\lib\site-packages\ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


 2 4 0.70 10000000000.00

In [6]:
# with open('heart_5fold_index.pkl', 'wb') as f:
#     pkl.dump([X_train, X_testN, y_train, y_testN, X_val, X_testO, y_val, y_testO], f)

In [7]:
pkl.dump(all_res_val, open('Heart_'+ privfeat +'_resVal_ordinary_1_KOC_5CV_CV.pkl', 'wb')) # wb denotes write bytestream
pkl.dump(all_res_test, open('Heart_'+ privfeat +'_resTest_ordinary_1_KOC_5CV_CV.pkl', 'wb')) # wb denotes write bytestream

In [8]:
resultsVal = pkl.load(open('Heart_'+ privfeat +'_resVal_ordinary_1_KOC_5CV_CV.pkl', 'rb')) # rb denotes read bytestream
resultsTest = pkl.load(open('Heart_'+ privfeat +'_resTest_ordinary_1_KOC_5CV_CV.pkl', 'rb')) # rb denotes read bytestream

In [9]:
all_nu = np.linspace(0.05, 0.7, 10)
all_gamma = np.logspace(-10, 10, 20) 

val_scores = np.zeros((2,5, all_nu.size, all_gamma.size))
test_scores = np.zeros((2,5, all_nu.size, all_gamma.size))

###    Load 5-fold CV files
with open('heart_5fold_index.pkl', 'rb') as f:
    X_train, X_testN, y_train, y_testN, X_val, X_testO, y_val, y_testO = pkl.load(f)
    
for index_index, index in enumerate(range(1, 3)):
    for f_index, f in enumerate(range(0, 5)):
        
#        print index,f
        X_vali = pd.concat([feature_space.ix[X_train[index][str(f)]][int(math.floor(len(X_train[index][str(f)])*0.8)):], feature_space.ix[X_val[index][str(f)]]], axis=0,sort=False)
        Y_vali = pd.concat([y_train[index][str(f)][int(math.floor(len(X_train[index][str(f)])*0.8)):], y_val[index][str(f)]], axis=0,sort=False)
        X_testi = pd.concat([feature_space.ix[X_testN[index][str(f)]], feature_space.ix[X_testO[index][str(f)]]], axis=0,sort=False)
        Y_testi = pd.concat([y_testN[index][str(f)], y_testO[index][str(f)]], axis=0,sort=False)
      
        for nu_index, nu in enumerate(resultsVal[index][f]):
            for gamma_index, gamma in enumerate(resultsVal[index][f][nu]): 

### Average Precision Score
                val_scores[index_index,f_index,nu_index,gamma_index] = average_precision_score(Y_vali, resultsVal[index][f][nu][gamma])
                test_scores[index_index,f_index,nu_index,gamma_index] = average_precision_score(Y_testi, resultsTest[index][f][nu][gamma])

#### For python 3
            print ('\r{0} {1} {2}'.format(index_index, f_index, nu_index, gamma_index), end='')
    
print ("\nmean_AUC_PRCurve",np.mean(val_scores),np.mean(test_scores))        

1 4 9
mean_AUC_PRCurve 0.3379043657891682 0.6286261019634476


In [10]:
fold_num = 4
print  ("max_value of class 1 in test_score for fold=%d is" %fold_num, max(test_scores[0,fold_num,:,:].flatten()))
temp_array1 = val_scores[0,fold_num,:,:]
i_max,j_max = np.unravel_index(temp_array1.argmax(), temp_array1.shape)
print ("Class 1 for fold=%d is" %fold_num, test_scores[0,fold_num, i_max,j_max])

print ("max_value of class 2 in test_score", max(test_scores[1,fold_num,:,:].flatten()))
temp_array2 = val_scores[1,fold_num,:,:]
i_max,j_max = np.unravel_index(temp_array2.argmax(), temp_array2.shape)
print ("Class 2  for fold=%d is" %fold_num, test_scores[1,fold_num, i_max,j_max])

max_value of class 1 in test_score for fold=4 is 0.8722555095024943
Class 1 for fold=4 is 0.8706169002771506
max_value of class 2 in test_score 0.769156170880259
Class 2  for fold=4 is 0.7657028455241435


In [11]:
# ax = plt.contourf(all_gamma, all_nu, test_scores[8,0,:,:])
# plt.xscale('log', basex=2)
# plt.yscale('log', basey=2)
# CS = plt.colorbar(ax)

In [12]:
val_opt_auc = np.zeros((2,5))
test_opt_auc = np.zeros((2,5))
val_opt_nu = np.zeros((2,5))
val_opt_gamma = np.zeros((2,5))
for index_index, index in enumerate(range(1, 3)):
    for f_index, f in enumerate(range(0, 5)):
        temp_arr = val_scores[index_index,f_index,:,:]
        i_max,j_max = np.unravel_index(temp_arr.argmax(), temp_arr.shape)
#         print "Val_max", val_scores[index_index,f_index,i_max,j_max]
#         print "Test_max", test_scores[index_index,f_index,i_max,j_max]
        val_opt_auc[index_index,f_index] = val_scores[index_index,f_index,i_max,j_max]
        test_opt_auc[index_index,f_index] = test_scores[index_index,f_index,i_max,j_max]
        val_opt_nu[index_index,f_index] = all_nu[i_max]
        val_opt_gamma[index_index,f_index] = all_gamma[j_max]
###         print "Val_max", np.amax(val_scores[index_index,f_index,:,:])
###         print "Test_max", np.amax(test_scores[index_index,f_index,:,:])

## convert your array into a dataframe
dfval = pd.DataFrame (val_opt_auc)
dftest = pd.DataFrame (test_opt_auc)
dfnu = pd.DataFrame (val_opt_nu)
dfgamma = pd.DataFrame (val_opt_gamma)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Final_Results_KOC_Heart_'+ privfeat +'.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
### Write optimum Parameteres
### Write Optimum Value corresponding to Optimum parameters
dfval.to_excel(writer, sheet_name='val_opt_KOC')
dftest.to_excel(writer, sheet_name='test_opt_KOC')
# dfnu.to_excel(writer, sheet_name='val_opt_KOC',startcol=9,startrow=0,index=False)
dfnu.to_excel(writer, sheet_name='nu_opt_KOC')
dfgamma.to_excel(writer, sheet_name='gamma_opt_KOC')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

### IKOC (KOC+)

In [13]:

all_res_val = {}
all_res_test = {}
X_train = {}
X_testN = {}
y_train = {}
y_testN = {}
X_val = {}  
X_testO = {}
y_val = {}
y_testO = {} 

##### Either run below commented lines to generate 5-fold indices or load the saving indices as below
with open('heart_5fold_index.pkl', 'rb') as f:
        X_train, X_testN, y_train, y_testN, X_val, X_testO, y_val, y_testO = pkl.load(f)
        
for index_index, index in enumerate(range(1, 3)):
    
        ##### Either run this to generate 5-fold indices or load the saving indices as before above for loop
        
#     target_files = labels.query('Class == {}'.format(index)).index   ### All Target Class Files wiith indices
#     test_indices = set(feature_space.index) - set(target_files) ### in Set for
#     test_indices = pd.Index(test_indices) ### Convert set to index
#     target_labels = labels.ix[target_files].Class == index ### Target data labels
#     Outlier_labels = labels.ix[test_indices].Class == index ### Target data labels

    
#     kf = KFold(n_splits=5) ### k-fold Cross-Validation
#     # For Target Files
#     fold = 0
#     X_train[index] = {}
#     X_testN[index] = {}
#     y_train[index] = {}
#     y_testN[index] = {}
#     for train_index, test_index in kf.split(target_files):
#         #print("TRAIN:", train_index, "TEST:", test_index)
#         X_train[index][str(fold)]=target_files[train_index]
#         X_testN[index][str(fold)]=target_files[test_index]
#         y_train[index][str(fold)]=target_labels[train_index]
#         y_testN[index][str(fold)]=target_labels[test_index]
#         fold = fold+1

#     # For Outlier Files
#     fold = 0
#     X_val[index] = {}  
#     X_testO[index] = {}
#     y_val[index] = {}
#     y_testO[index] = {}    
#     for train_index, test_index in kf.split(test_indices):
# #         print("TRAIN:", train_index, "TEST:", test_index)
#         X_val[index][str(fold)]=test_indices[train_index]
#         X_testO[index][str(fold)]=test_indices[test_index]
#         y_val[index][str(fold)]=Outlier_labels[train_index]
#         y_testO[index][str(fold)]=Outlier_labels[test_index] 
#         fold=fold+1

        
### Cocatenate and Prepare final data
    all_res_val[index] = {}
    all_res_test[index] = {}
    for f_index, f in enumerate(range(0, 5)):
        all_res_val[index][f_index] = {}
        all_res_test[index][f_index] = {}
#        print index,f
        X_vali = pd.concat([feature_space.ix[X_train[index][str(f)]][int(math.floor(len(X_train[index][str(f)])*0.8)):], feature_space.ix[X_val[index][str(f)]]], axis=0,sort=False)
        Xprev_vali = pd.concat([privileged_space.ix[X_train[index][str(f)]][int(math.floor(len(X_train[index][str(f)])*0.8)):], privileged_space.ix[X_val[index][str(f)]]], axis=0,sort=False)
        Y_vali = pd.concat([y_train[index][str(f)][int(math.floor(len(X_train[index][str(f)])*0.8)):], y_val[index][str(f)]], axis=0,sort=False)
        X_testi = pd.concat([feature_space.ix[X_testN[index][str(f)]], feature_space.ix[X_testO[index][str(f)]]], axis=0,sort=False)
        Y_testi = pd.concat([y_testN[index][str(f)], y_testO[index][str(f)]], axis=0,sort=False)
#### Take 80% of samples for training and keep 20% samples for validation
        train_id = X_train[index][str(f)][:int(math.floor(len(X_train[index][str(f)])*0.8))]
        train_data = feature_space.ix[train_id]
        privileged_train = privileged_space.ix[train_id]
        train_label = y_train[index][str(f)][:int(math.floor(len(X_train[index][str(f)])*0.8))]
        ### Parameters
        all_nu_priv = np.linspace(0.05, 0.7, 10) ### Privileged Regularization Parameter (mu as per paper)
        all_priv_gamma = np.logspace(-10, 10, 20) 
    
        all_nu = np.linspace(0.05, 0.7, 10) ### Regularization parameter (C = 1/nu*no. of samples)
        all_gamma = np.logspace(-10, 10, 20) # Power of 10 is called logspace. -10 is start point, 10 is last point ans 20 points will be generated
      
        for nu1 in all_nu:
            all_res_val[index][f_index][nu1] = {}
            all_res_test[index][f_index][nu1] = {}
            for gamma1 in all_gamma: 
                all_res_val[index][f_index][nu1][gamma1] = {}
                all_res_test[index][f_index][nu1][gamma1] = {}
            #### Generate feature kernel function using gamma1     
                features_kernel = partial(rbf_kernel, gamma=gamma1)
                for nu in all_nu_priv:
                    all_res_val[index][f_index][nu1][gamma1][nu] = {}
                    all_res_test[index][f_index][nu1][gamma1][nu] = {}
                    for gamma in all_priv_gamma:
                        privileged_kernel = partial(rbf_kernel, gamma=gamma)
                        model = IKOC(nu=nu1, features_kernel=features_kernel, privileged_kernel=privileged_kernel, privileged_regularization=nu)
                        model.fit(train_data.as_matrix(), privileged_train.as_matrix().reshape(-1, 1))
                        pred_val = model.decision_function(X_vali)
                        all_res_val[index][f_index][nu1][gamma1][nu][gamma] = pred_val              
                        pred_test = model.decision_function(X_testi)
                        all_res_test[index][f_index][nu1][gamma1][nu][gamma] = pred_test             
#                         print '\r {0} {1} {2:.2f} {3:.2f} {4:.2f} {5:.2f}'.format(index, f, nu1, gamma1, nu, gamma),
                        print ('\r {0} {1} {2:.2f} {3:.2f} {4:.2f} {5:.2f}'.format(index, f, nu1, gamma1, nu, gamma), end='')

C:\Users\Workstation\Anaconda2\lib\site-packages\ipykernel_launcher.py:96: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


 2 4 0.70 10000000000.00 0.70 10000000000.00

In [14]:
pkl.dump(all_res_val, open('Heart_'+ privfeat +'_resVal_3_KOC+_5CV_CV.pkl', 'wb')) # wb denotes write bytestream
pkl.dump(all_res_test, open('Heart_'+ privfeat +'_resTest_3_KOC+_5CV_CV.pkl', 'wb')) # wb denotes write bytestream

In [15]:
resultsVal = pkl.load(open('Heart_'+ privfeat +'_resVal_3_KOC+_5CV_CV.pkl', 'rb')) # rb denotes read bytestream
resultsTest = pkl.load(open('Heart_'+ privfeat +'_resTest_3_KOC+_5CV_CV.pkl', 'rb')) # rb denotes read bytestream

In [16]:
### Parameters
all_nu_priv = np.linspace(0.05, 0.7, 10) 
all_priv_gamma = np.logspace(-10, 10, 20)
    
all_nu = np.linspace(0.05, 0.7, 10)
all_gamma = np.logspace(-10, 10, 20) 
        
val_scores = np.zeros((2, 5, all_nu.size, all_gamma.size, all_nu_priv.size, all_priv_gamma.size))
test_scores = np.zeros((2, 5, all_nu.size, all_gamma.size, all_nu_priv.size, all_priv_gamma.size))

###    Load 5-fold CV files
with open('heart_5fold_index.pkl', 'rb') as f:
    X_train, X_testN, y_train, y_testN, X_val, X_testO, y_val, y_testO = pkl.load(f)
    
for index_index, index in enumerate(range(1, 3)):
    for f_index, f in enumerate(range(0, 5)):
        
#        print index,f
        X_vali = pd.concat([feature_space.ix[X_train[index][str(f)]][int(math.floor(len(X_train[index][str(f)])*0.8)):], feature_space.ix[X_val[index][str(f)]]], axis=0,sort=False)
        Y_vali = pd.concat([y_train[index][str(f)][int(math.floor(len(X_train[index][str(f)])*0.8)):], y_val[index][str(f)]], axis=0,sort=False)
        X_testi = pd.concat([feature_space.ix[X_testN[index][str(f)]], feature_space.ix[X_testO[index][str(f)]]], axis=0,sort=False)
        Y_testi = pd.concat([y_testN[index][str(f)], y_testO[index][str(f)]], axis=0,sort=False)
      
        for nu1_index, nu1 in enumerate(resultsVal[index][f_index]):
            for gamma1_index, gamma1 in enumerate(resultsVal[index][f_index][nu1]):
                for nu_index, nu in enumerate(resultsVal[index][f_index][nu1][gamma1]):
                    for gamma_index, gamma in enumerate(resultsVal[index][f_index][nu1][gamma1][nu]): 
#                         print resultsVal[index][f_index][nu1][gamma1][nu][gamma]
### Average Precision Score
                        val_scores[index_index,f_index,nu1_index, gamma1_index, nu_index,gamma_index] = average_precision_score(Y_vali, resultsVal[index][f_index][nu1][gamma1][nu][gamma])
                        test_scores[index_index,f_index,nu1_index, gamma1_index, nu_index,gamma_index] = average_precision_score(Y_testi, resultsTest[index][f_index][nu1][gamma1][nu][gamma])

#### For python 3
                        print ('\r{0} {1} {2} {3} {4} {5}'.format(index_index, f_index, nu1_index, gamma1_index, nu_index, gamma_index), end='')


print ("\nmean_AUC_PRCurve",np.mean(val_scores),np.mean(test_scores))       

1 4 9 19 9 19
mean_AUC_PRCurve 0.18267990870927384 0.48723333920806405


In [17]:
fold_num = 4
print  ("max_value of class 1 in test_score for fold=%d is" %fold_num, max(test_scores[0,fold_num,:,:,:,:].flatten()))
temp_array1 = val_scores[0,fold_num,:,:,:,:]
i_max,j_max, k_max, l_max = np.unravel_index(temp_array1.argmax(), temp_array1.shape)
print ("Class 1 for fold=%d is" %fold_num, test_scores[0,fold_num, i_max,j_max, k_max, l_max])

print ("max_value of class 2 in test_score for fold=%d is" %fold_num, max(test_scores[1,fold_num,:,:,:,:].flatten()))
temp_array2 = val_scores[1,fold_num,:,:,:,:]
i_max,j_max, k_max, l_max = np.unravel_index(temp_array2.argmax(), temp_array2.shape)
print ("Class 2  for fold=%d is" %fold_num, test_scores[1,fold_num, i_max,j_max, k_max, l_max])

max_value of class 1 in test_score for fold=4 is 0.9555535803923909
Class 1 for fold=4 is 0.9080602275599117
max_value of class 2 in test_score for fold=4 is 0.917280592457977
Class 2  for fold=4 is 0.8619943115468185


In [18]:
### Select optimal parameter and save corresponding results and parameters in excel sheet

from openpyxl import load_workbook

val_opt_auc = np.zeros((2,5))
test_opt_auc = np.zeros((2,5))
val_opt_nu = np.zeros((2,5))
val_opt_gamma = np.zeros((2,5))
val_opt_priv_nu = np.zeros((2,5))
val_opt_priv_gamma = np.zeros((2,5))
for index_index, index in enumerate(range(1, 3)):
    for f_index, f in enumerate(range(0, 5)):
        temp_arr = val_scores[index_index,f_index,:,:,:,:]
        i_max,j_max,k_max,l_max = np.unravel_index(temp_arr.argmax(), temp_arr.shape)
#         print "Val_max", val_scores[index_index,f_index,i_max,j_max]
#         print "Test_max", test_scores[index_index,f_index,i_max,j_max]
        val_opt_auc[index_index,f_index] = val_scores[index_index,f_index,i_max,j_max,k_max,l_max]
        test_opt_auc[index_index,f_index] = test_scores[index_index,f_index,i_max,j_max,k_max,l_max]
        val_opt_nu[index_index,f_index] = all_nu[i_max]
        val_opt_gamma[index_index,f_index] = all_gamma[j_max]
        val_opt_priv_nu[index_index,f_index] = all_nu_priv[k_max]
        val_opt_priv_gamma[index_index,f_index] = all_priv_gamma[l_max]
###         print "Val_max", np.amax(val_scores[index_index,f_index,:,:])
###         print "Test_max", np.amax(test_scores[index_index,f_index,:,:])

## convert your array into a dataframe
dfval = pd.DataFrame (val_opt_auc)
dftest = pd.DataFrame (test_opt_auc)
dfnu = pd.DataFrame (val_opt_nu)
dfgamma = pd.DataFrame (val_opt_gamma)
dfprivnu = pd.DataFrame (val_opt_priv_nu)
dfprivgamma = pd.DataFrame (val_opt_priv_gamma)

### Load existing excel file
book = load_workbook('Final_Results_KOC_Heart_'+ privfeat +'.xlsx')
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Final_Results_KOC_Heart_'+ privfeat +'.xlsx', engine='openpyxl')
writer.book = book

# Convert the dataframe to an XlsxWriter Excel object.
### Write Optimum Value corresponding to Optimum parameters
dfval.to_excel(writer, sheet_name='3val_opt_KOC+')
dftest.to_excel(writer, sheet_name='3test_opt_KOC+')
### Write optimum Parameteres
# dfnu.to_excel(writer, sheet_name='nu_opt',startcol=9,startrow=0)
dfnu.to_excel(writer, sheet_name='3nu_opt+')
dfgamma.to_excel(writer, sheet_name='3gamma_opt+')
dfprivnu.to_excel(writer, sheet_name='3privnu_opt+')
dfprivgamma.to_excel(writer, sheet_name='3privgamma_opt+')

# Close the Pandas Excel writer and output the Excel file.
writer.save() 
writer.close()